## Accessing data with an http or opendap URL is very easy

In [1]:
import xarray as xr

In [2]:
# Direct access data from kage:
# Note, use your browser to explore http://kage.ldeo.columbia.edu/data/pdsi-spei/

url = 'http://kage.ldeo.columbia.edu/data/pdsi-spei/pdsi/GFDL_ESM2G/pdsi_all.nc#mode=bytes'
ds = xr.open_dataset(url,decode_times=False)
print(ds)
#ds.pdsi_all.mean('T').plot()

<xarray.Dataset>
Dimensions:   (T: 2400, ensemble: 1, lat: 75, lon: 144)
Coordinates:
  * lon       (lon) float32 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lat       (lat) float32 -59.66 -57.64 -55.62 -53.6 ... 85.96 87.98 89.49
  * ensemble  (ensemble) float32 1.0
  * T         (T) float32 0.5 1.5 2.5 3.5 ... 2.398e+03 2.398e+03 2.4e+03
Data variables:
    pdsi_all  (T, ensemble, lat, lon) float64 ...


In [3]:
# Direct access CMIP6 from mary:

url = 'http://mary.ldeo.columbia.edu/CMIP6/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/Amon/clt/gn/v20181214/\
clt_Amon_BCC-ESM1_historical_r1i1p1f1_gn_185001-201412.nc#mode=bytes'

xr.open_dataset(url)

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 64, lon: 128, time: 1980)
Coordinates:
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    clt        (time, lat, lon) float32 ...
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          Standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  2110.0
    comment:                The model integration starts from the piControl e...
    contact:                Dr. Tongwen Wu(twwu@cma.gov.cn)
    creation_date:          2018-12-14T05:25:13Z
    data_specs_version:     01.00.27
    description:            DECK: historical
    experiment:             all-forcing simulation of the recent past
    experiment_id:          historical
    external_variables:     areacella
    forcing_index:          1
    frequency:              mon
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.BCC.BCC-ESM1...
    grid:                   T42
    grid_label:             gn
    history:                2018-12-14T05:25:12Z ; CMOR rewrote data to be co...
    initialization_index:   1
    institution:            Beijing Climate Center, Beijing 100081, China
    institution_id:         BCC
    mip_era:                CMIP6
    nominal_resolution:     250 km
    parent_activity_id:     CMIP
    parent_experiment_id:   piControl
    parent_mip_era:         CMIP6
    parent_source_id:       BCC-ESM1
    parent_time_units:      days since 1850-01-01
    parent_variant_label:   r1i1p1f1
    physics_index:          1
    product:                model-output
    realization_index:      1
    realm:                  atmos
    references:             Model described by Tongwen Wu et al. (JGR 2013; J...
    run_variant:            forcing: greenhouse gases,aerosol emission,solar ...
    source:                 BCC-ESM 1 (2017):   aerosol: none  atmos: BCC_AGC...
    source_id:              BCC-ESM1
    source_type:            AER AOGCM CHEM
    sub_experiment:         none
    sub_experiment_id:      none
    table_id:               Amon
    table_info:             Creation Date:(30 July 2018) MD5:e53ff52009d0b97d...
    title:                  BCC-ESM1 output prepared for CMIP6
    tracking_id:            hdl:21.14100/417eb9bd-0df7-4dcc-a325-e96dab8c44b3
    variable_id:            clt
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by BCC is licensed unde...
    cmor_version:           3.3.2

In [4]:
# Can also use an OPeNDAP url:

url = 'https://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/ssp370/r1i1p1f1/day/tasmax/gn/v20210323/tasmax_day_TaiESM1_ssp370_r1i1p1f1_gn_20150101-20241231.nc'
xr.open_dataset(url)

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 192, lon: 288, time: 3650)
Coordinates:
  * time       (time) object 2015-01-01 12:00:00 ... 2024-12-31 12:00:00
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tasmax     (time, lat, lon) float32 ...
Attributes:
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     ScenarioMIP AerChemMIP
    branch_method:                   Hybrid-restart from year 2015-01-01 of h...
    branch_time:                     0.0
    branch_time_in_child:            0.0
    branch_time_in_parent:           60225.0
    contact:                         Dr. Wei-Liang Lee (leelupin@gate.sinica....
    creation_date:                   2021-03-23T02:39:58Z
    data_specs_version:              01.00.31
    experiment:                      gap-filling scenario reaching 7.0 based ...
    experiment_id:                   ssp370
    external_variables:              areacella
    forcing_index:                   1
    frequency:                       day
    further_info_url:                https://furtherinfo.es-doc.org/CMIP6.AS-...
    grid:                            finite-volume grid with 0.9x1.25 degree ...
    grid_label:                      gn
    history:                         2021-03-23T02:39:58Z ; CMOR rewrote data...
    initialization_index:            1
    institution:                     Research Center for Environmental Change...
    institution_id:                  AS-RCEC
    mip_era:                         CMIP6
    model_id:                        TaiESM1
    nominal_resolution:              100 km
    parent_activity_id:              CMIP
    parent_experiment_id:            historical
    parent_mip_era:                  CMIP6
    parent_source_id:                TaiESM1
    parent_sub_experiment_id:        none
    parent_time_units:               days since 1850-01-01
    parent_variant_label:            r1i1p1f1
    physics_index:                   1
    product:                         model-output
    realization_index:               1
    realm:                           atmos
    references:                      https://doi.org/10.5194/gmd-2019-377
    run_variant:                     N/A
    source:                          TaiESM 1.0 (2018): \naerosol: SNAP (same...
    source_id:                       TaiESM1
    source_type:                     AOGCM AER BGC
    sub_experiment:                  none
    sub_experiment_id:               none
    table_id:                        day
    table_info:                      Creation Date:(24 July 2019) MD5:b0d7711...
    title:                           TaiESM1 output prepared for CMIP6
    variable_id:                     tasmax
    variant_label:                   r1i1p1f1
    license:                         CMIP6 model data produced by NCC is lice...
    cmor_version:                    3.5.0
    tracking_id:                     hdl:21.14100/53e1e966-7957-4b7b-b1cf-76d...
    DODS_EXTRA.Unlimited_Dimension:  time

In [5]:
# Globus - NOPE, must set up endpoints, etc.

url = 'globus:415a6320-e49c-11e5-9798-22000b9da45e/css03_data/CMIP6/ScenarioMIP/IPSL/IPSL-CM6A-LR/ssp370/r6i1p1f1/day\
/tasmax/gr/v20190614/tasmax_day_IPSL-CM6A-LR_ssp370_r6i1p1f1_gr_20150101-21001231.nc'
# xr.open_dataset(url)

In [6]:
# Google Cloud Store, Pangeo zarr collection

path = 'CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp585/r2i1p1f1/day/va/gn/v20190710'

url = f'https://cmip6.storage.googleapis.com/{path}'
ds = xr.open_zarr(url,consolidated=True)
print('size of dataset:',ds.nbytes/1e9,'G')
ds

size of dataset: 18.527722504 G


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lon: 192, plev: 8, time: 31411)
Coordinates:
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * time       (time) datetime64[ns] 2015-01-01T12:00:00 ... 2100-12-31T12:00:00
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(15706, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    va         (time, plev, lat, lon) float32 dask.array<chunksize=(142, 8, 96, 192), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    cmor_version:           3.5.0
    contact:                cmip6-mpi-esm@dkrz.de
    creation_date:          2019-11-12T23:11:36Z
    data_specs_version:     01.00.30
    experiment:             update of RCP8.5 based on SSP5
    experiment_id:          ssp585
    external_variables:     areacella
    forcing_index:          1
    frequency:              day
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.MPI-M.MPI-ES...
    grid:                   gn
    grid_label:             gn
    history:                2019-11-12T23:11:36Z ; CMOR rewrote data to be co...
    initialization_index:   1
    institution:            Max Planck Institute for Meteorology, Hamburg 201...
    institution_id:         MPI-M
    license:                CMIP6 model data produced by MPI-M is licensed un...
    mip_era:                CMIP6
    netcdf_tracking_ids:    hdl:21.14100/98dc2553-0e26-46df-9115-6bbc173deef6...
    nominal_resolution:     250 km
    parent_activity_id:     CMIP
    parent_experiment_id:   historical
    parent_mip_era:         CMIP6
    parent_source_id:       MPI-ESM1-2-LR
    parent_time_units:      days since 1850-1-1 00:00:00
    parent_variant_label:   r2i1p1f1
    physics_index:          1
    product:                model-output
    project_id:             CMIP6
    realization_index:      2
    realm:                  atmos
    references:             MPI-ESM: Mauritsen, T. et al. (2019), Development...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    source_id:              MPI-ESM1-2-LR
    source_type:            AOGCM
    status:                 2021-09-22;created; by gcs.cmip6.ldeo@gmail.com
    sub_experiment:         none
    sub_experiment_id:      none
    table_id:               day
    table_info:             Creation Date:(09 May 2019) MD5:e6ef8ececc8f33864...
    title:                  MPI-ESM1-2-LR output prepared for CMIP6
    tracking_id:            hdl:21.14100/98dc2553-0e26-46df-9115-6bbc173deef6
    variable_id:            va
    variant_label:          r2i1p1f1
    version_id:             v20190710

In [7]:
# Amazon S3, Pangeo zarr collection (mirror of GCS)

path = 'CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp585/r2i1p1f1/day/va/gn/v20190710'

url = f'https://cmip6-pds.s3.amazonaws.com/{path}'
ds = xr.open_zarr(url,consolidated=True)
print('size of dataset:',ds.nbytes/1e9,'G')
ds

size of dataset: 18.527722504 G


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lon: 192, plev: 8, time: 31411)
Coordinates:
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * time       (time) datetime64[ns] 2015-01-01T12:00:00 ... 2100-12-31T12:00:00
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(15706, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    va         (time, plev, lat, lon) float32 dask.array<chunksize=(142, 8, 96, 192), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    cmor_version:           3.5.0
    contact:                cmip6-mpi-esm@dkrz.de
    creation_date:          2019-11-12T23:11:36Z
    data_specs_version:     01.00.30
    experiment:             update of RCP8.5 based on SSP5
    experiment_id:          ssp585
    external_variables:     areacella
    forcing_index:          1
    frequency:              day
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.MPI-M.MPI-ES...
    grid:                   gn
    grid_label:             gn
    history:                2019-11-12T23:11:36Z ; CMOR rewrote data to be co...
    initialization_index:   1
    institution:            Max Planck Institute for Meteorology, Hamburg 201...
    institution_id:         MPI-M
    license:                CMIP6 model data produced by MPI-M is licensed un...
    mip_era:                CMIP6
    netcdf_tracking_ids:    hdl:21.14100/98dc2553-0e26-46df-9115-6bbc173deef6...
    nominal_resolution:     250 km
    parent_activity_id:     CMIP
    parent_experiment_id:   historical
    parent_mip_era:         CMIP6
    parent_source_id:       MPI-ESM1-2-LR
    parent_time_units:      days since 1850-1-1 00:00:00
    parent_variant_label:   r2i1p1f1
    physics_index:          1
    product:                model-output
    project_id:             CMIP6
    realization_index:      2
    realm:                  atmos
    references:             MPI-ESM: Mauritsen, T. et al. (2019), Development...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    source_id:              MPI-ESM1-2-LR
    source_type:            AOGCM
    status:                 2021-09-22;created; by gcs.cmip6.ldeo@gmail.com
    sub_experiment:         none
    sub_experiment_id:      none
    table_id:               day
    table_info:             Creation Date:(09 May 2019) MD5:e6ef8ececc8f33864...
    title:                  MPI-ESM1-2-LR output prepared for CMIP6
    tracking_id:            hdl:21.14100/98dc2553-0e26-46df-9115-6bbc173deef6
    variable_id:            va
    variant_label:          r2i1p1f1
    version_id:             v20190710

In [8]:
# Amazon S3, GFDL netcdf collection

# can open a single file like this, but there is a better method to read from the S3 file system directly, see below

#url = 'https://esgf-world.s3.amazonaws.com/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Lmon/cLeaf/gr/v20190118/cLeaf_Lmon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gr_185001-214912.nc#mode=bytes'
#ds = xr.open_dataset(url, decode_coords=False)
#ds

## We can also instantiate a file system

In [9]:
import xarray as xr
import fsspec

fs_http = fsspec.filesystem('http')
files = fs_http.glob('http://kage.ldeo.columbia.edu/data/ERA5/monthly/single_level/*/2m_*.nc')
files

['http://kage.ldeo.columbia.edu/data/ERA5/monthly/single_level/y1950-1978/2m_dewpoint_temperature.nc',
 'http://kage.ldeo.columbia.edu/data/ERA5/monthly/single_level/y1950-1978/2m_temperature.nc',
 'http://kage.ldeo.columbia.edu/data/ERA5/monthly/single_level/y1979-2020/2m_dewpoint_temperature.nc',
 'http://kage.ldeo.columbia.edu/data/ERA5/monthly/single_level/y1979-2020/2m_temperature.nc']

In [10]:
urls = [ file + '#mode=bytes' for file in files]
ds= xr.open_mfdataset(urls)
ds

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 840)
Coordinates:
    expver     int32 1
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2019-12-01
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(348, 721, 1440), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(348, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-02-22 20:19:09 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [11]:
# can browse ftp sites
fs_ftp = fsspec.filesystem('ftp', host='ftp.cdc.noaa.gov') #, port=port, username=user, password=pw)
fs_ftp.glob('/Projects/Datasets/*')

['/Projects/Datasets/COBE',
 '/Projects/Datasets/COBE2',
 '/Projects/Datasets/CarbonTracker',
 '/Projects/Datasets/README',
 '/Projects/Datasets/Timeseries',
 '/Projects/Datasets/cmap',
 '/Projects/Datasets/cpc_global_precip',
 '/Projects/Datasets/cpc_global_temp',
 '/Projects/Datasets/cpc_us_hour_precip',
 '/Projects/Datasets/cpc_us_precip',
 '/Projects/Datasets/cpcsoil',
 '/Projects/Datasets/cru',
 '/Projects/Datasets/dai_pdsi',
 '/Projects/Datasets/ghcncams',
 '/Projects/Datasets/ghcngridded',
 '/Projects/Datasets/gistemp',
 '/Projects/Datasets/godas',
 '/Projects/Datasets/gpcc',
 '/Projects/Datasets/gpcp',
 '/Projects/Datasets/icoads',
 '/Projects/Datasets/icoads2.5',
 '/Projects/Datasets/interp_OLR',
 '/Projects/Datasets/jmatemp',
 '/Projects/Datasets/kaplan_sst',
 '/Projects/Datasets/mlost',
 '/Projects/Datasets/mlostv3b',
 '/Projects/Datasets/mount_check_file',
 '/Projects/Datasets/msu',
 '/Projects/Datasets/ncep',
 '/Projects/Datasets/ncep.marine',
 '/Projects/Datasets/ncep.pac

In [12]:
# can directly read in single ftp files WITHOUT downloading first
import urllib
import io

url = 'ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/surface/air.sig995.mon.mean.nc'
req = urllib.request.Request(url)

with urllib.request.urlopen(req) as resp:
    ds = xr.open_dataset(io.BytesIO(resp.read()))
    
ds

<xarray.Dataset>
Dimensions:  (lat: 73, lon: 144, time: 884)
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1948-01-01 1948-02-01 ... 2021-08-01
Data variables:
    air      (time, lat, lon) float32 ...
Attributes:
    description:    Data from NCEP initialized reanalysis (4x/day).  These ar...
    platform:       Model
    Conventions:    COARDS
    NCO:            20121012
    history:        Thu May  4 20:11:16 2000: ncrcat -d time,0,623 /Datasets/...
    title:          monthly mean air.sig995 from the NCEP Reanalysis
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [13]:
# Google Cloud Store Example
import gcsfs

fs_GCS = gcsfs.GCSFileSystem(token='anon',access='read_only')

fs_GCS.ls('cmip6')  # just put the bucket name here

['cmip6/CMIP3',
 'cmip6/CMIP5',
 'cmip6/CMIP6',
 'cmip6/DCPP',
 'cmip6/GFDL_CM2_6',
 'cmip6/cmip6-zarr-consolidated-stores-noQC.csv',
 'cmip6/cmip6-zarr-consolidated-stores-noQC.jsonl',
 'cmip6/cmip6-zarr-consolidated-stores.csv',
 'cmip6/compare',
 'cmip6/gfdl_cm2_6.json',
 'cmip6/glade-cmip6.csv.gz',
 'cmip6/gsutil_test_filelz288lpb.bin',
 'cmip6/pangeo-cmip3.csv',
 'cmip6/pangeo-cmip3.json',
 'cmip6/pangeo-cmip5.csv',
 'cmip6/pangeo-cmip5.json',
 'cmip6/pangeo-cmip6-noQC.csv',
 'cmip6/pangeo-cmip6-noQC.json',
 'cmip6/pangeo-cmip6.csv',
 'cmip6/pangeo-cmip6.json',
 'cmip6/tracmip',
 'cmip6/tracmip.csv',
 'cmip6/tracmip.json']

In [14]:
# Amazon S3 Example
import s3fs

fs_S3 = s3fs.S3FileSystem(anon=True)

fs_S3.ls('cmip6-pds')  # just put the bucket name here

['cmip6-pds/CMIP3',
 'cmip6-pds/CMIP5',
 'cmip6-pds/CMIP6',
 'cmip6-pds/DCPP',
 'cmip6-pds/GFDL_CM2_6',
 'cmip6-pds/compare',
 'cmip6-pds/gfdl_cm2_6.json',
 'cmip6-pds/gsutil_test_filelz288lpb.bin',
 'cmip6-pds/index.html',
 'cmip6-pds/pangeo-cmip3.csv',
 'cmip6-pds/pangeo-cmip3.json',
 'cmip6-pds/pangeo-cmip5.csv',
 'cmip6-pds/pangeo-cmip5.json',
 'cmip6-pds/pangeo-cmip6-noQC.csv',
 'cmip6-pds/pangeo-cmip6-noQC.csv.gz',
 'cmip6-pds/pangeo-cmip6-noQC.json',
 'cmip6-pds/pangeo-cmip6.csv',
 'cmip6-pds/pangeo-cmip6.json',
 'cmip6-pds/tracmip',
 'cmip6-pds/tracmip.csv',
 'cmip6-pds/tracmip.json']

In [15]:
# Amazon S3, GFDL netcdf collection  (dataset may consist of multiple netcdf files)

fs_GFDL = s3fs.S3FileSystem(anon=True)

path = 'CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp585/r2i1p1f1/day/va/gn/v20190710'

s3path = 's3://esgf-world/'+path+'/*.nc'
fs_GFDL.glob(s3path)

['esgf-world/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp585/r2i1p1f1/day/va/gn/v20190710/va_day_MPI-ESM1-2-LR_ssp585_r2i1p1f1_gn_20150101-20341231.nc',
 'esgf-world/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp585/r2i1p1f1/day/va/gn/v20190710/va_day_MPI-ESM1-2-LR_ssp585_r2i1p1f1_gn_20350101-20541231.nc',
 'esgf-world/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp585/r2i1p1f1/day/va/gn/v20190710/va_day_MPI-ESM1-2-LR_ssp585_r2i1p1f1_gn_20550101-20741231.nc',
 'esgf-world/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp585/r2i1p1f1/day/va/gn/v20190710/va_day_MPI-ESM1-2-LR_ssp585_r2i1p1f1_gn_20750101-20941231.nc',
 'esgf-world/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp585/r2i1p1f1/day/va/gn/v20190710/va_day_MPI-ESM1-2-LR_ssp585_r2i1p1f1_gn_20950101-21001231.nc']

In [16]:
fobj = [fs_GFDL.open(f) for f in fs_GFDL.glob(s3path)]
ds = xr.open_mfdataset(fobj, data_vars='minimal', 
                        use_cftime=True, join='exact', combine='nested', concat_dim='time')
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lon: 192, plev: 8, time: 31411)
Coordinates:
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(7305, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
    va         (time, plev, lat, lon) float32 dask.array<chunksize=(7305, 8, 96, 192), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   [60265.]
    branch_time_in_parent:  [60265.]
    contact:                cmip6-mpi-esm@dkrz.de
    creation_date:          2019-11-12T23:11:36Z
    data_specs_version:     01.00.30
    experiment:             update of RCP8.5 based on SSP5
    experiment_id:          ssp585
    external_variables:     areacella
    forcing_index:          [1]
    frequency:              day
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.MPI-M.MPI-ES...
    grid:                   gn
    grid_label:             gn
    history:                2019-11-12T23:11:36Z ; CMOR rewrote data to be co...
    initialization_index:   [1]
    institution:            Max Planck Institute for Meteorology, Hamburg 201...
    institution_id:         MPI-M
    mip_era:                CMIP6
    nominal_resolution:     250 km
    parent_activity_id:     CMIP
    parent_experiment_id:   historical
    parent_mip_era:         CMIP6
    parent_source_id:       MPI-ESM1-2-LR
    parent_time_units:      days since 1850-1-1 00:00:00
    parent_variant_label:   r2i1p1f1
    physics_index:          [1]
    product:                model-output
    project_id:             CMIP6
    realization_index:      [2]
    realm:                  atmos
    references:             MPI-ESM: Mauritsen, T. et al. (2019), Development...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    source_id:              MPI-ESM1-2-LR
    source_type:            AOGCM
    sub_experiment:         none
    sub_experiment_id:      none
    table_id:               day
    table_info:             Creation Date:(09 May 2019) MD5:e6ef8ececc8f33864...
    title:                  MPI-ESM1-2-LR output prepared for CMIP6
    variable_id:            va
    variant_label:          r2i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/98dc2553-0e26-46df-9115-6bbc173deef6